<a href="https://colab.research.google.com/github/smm0320-cyber/Proyecto/blob/main/Rummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  RUMMY (Trio)
Integrantes:

*  Sebastian Mejia
* Santiago Becerra
* Jhon Jairo Sons
* Victoria Herrera

In [ ]:
import random

# --------------------------------
# Utilidades de cartas y formato
# --------------------------------

def crear_mazo():
    """
    Crea un mazo estándar de 52 cartas como strings 'valor+palo', lo baraja y lo retorna.

    """
    valores = ['A'] + [str(n) for n in range(2, 11)] + ['J', 'Q', 'K']
    palos = ['♠','♥','♦','♣']
    mazo = []
    for v in valores:
        for p in palos:
            mazo.append(v + p)  # ejemplo: "7H", "10C", "QS"
    random.shuffle(mazo)
    return mazo

def obtener_valor(carta):
    """Devuelve el valor de la carta (sin el palo). '10C'->'10', 'QH'->'Q'."""
    return carta[:-1]

def mano_a_str(mano):
    """Devuelve la mano enumerada: [0] 7H, [1] 10C, ..."""
    partes = []
    for i in range(len(mano)):
        partes.append("[" + str(i) + "] " + mano[i])
    return ", ".join(partes)

# -----------------------------
# Comprobaciones / acciones
# -----------------------------

def tiene_trio(mano):
    """
    True si hay tres (o más) cartas del mismo VALOR en la mano.
    Sin diccionarios: cuenta con bucles anidados (n es pequeño: 3-4).
    """
    for i in range(len(mano)):
        v = obtener_valor(mano[i])
        conteo = 0
        for j in range(len(mano)):
            if obtener_valor(mano[j]) == v:
                conteo += 1
        if conteo >= 3:
            return True
    return False

def robar_de_mazo(mano, mazo):
    """Mueve la carta del tope del mazo (última) a la mano, si hay."""
    if len(mazo) > 0:
        mano.append(mazo.pop())
        return True
    return False

def robar_de_descarte(mano, descarte):
    """Mueve la carta del tope del descarte (última) a la mano, si hay."""
    if len(descarte) > 0:
        mano.append(descarte.pop())
        return True
    return False

def descartar(mano, descarte, indice):
    """Mueve mano[indice] al tope del descarte."""
    carta = mano.pop(indice)
    descarte.append(carta)

def contar_valores(mano):
    """
    Cuenta valores usando solo listas.
    Retorna (valores[], conteos[]) paralelas.
    """
    valores = []
    conteos = []
    for c in mano:
        v = obtener_valor(c)
        pos = -1
        for k in range(len(valores)):
            if valores[k] == v:
                pos = k
                break
        if pos == -1:
            valores.append(v)
            conteos.append(1)
        else:
            conteos[pos] += 1
    return valores, conteos

# -------------
# Turnos
# -------------

def turno_jugador(mano_j, mazo, descarte):
    print("\n--- TU TURNO ---")
    print("Tu mano: " + mano_a_str(mano_j))
    print("Tope descarte: " + (descarte[-1] if len(descarte) > 0 else "(vacío)"))

    # (1) Elegir de dónde robar con validación de disponibilidad
    while True:
        opcion = input("Robar de 'mazo' o 'descarte'? ").strip().lower()
        if opcion == "mazo":
            if robar_de_mazo(mano_j, mazo):
                print("Robaste del mazo: " + mano_j[-1])
                break
            else:
                print("El mazo está vacío.")
        elif opcion == "descarte":
            if robar_de_descarte(mano_j, descarte):
                print("Tomaste del descarte: " + mano_j[-1])
                break
            else:
                print("El descarte está vacío.")
        else:
            print("Entrada inválida. Escribe 'mazo' o 'descarte'.")

    # (2) ¿Ganó al robar?
    if tiene_trio(mano_j):
        print("¡Formaste un TRÍO! 🎉")
        return True

    # (3) Descartar por índice válido
    print("Tu mano ahora: " + mano_a_str(mano_j))
    while True:
        txt = input("Índice a descartar (0.." + str(len(mano_j)-1) + "): ").strip()
        if txt.isdigit():
            i = int(txt)
            if 0 <= i < len(mano_j):
                descartar(mano_j, descarte, i)
                print("Descartaste: " + descarte[-1])
                break
            else:
                print("Índice fuera de rango.")
        else:
            print("Debes escribir un número.")

    return False

def turno_computadora(mano_pc, mazo, descarte):
    print("\n--- TURNO DE LA PC ---")
    print("(PC) mano: " + mano_a_str(mano_pc))
    print("(PC) Tope descarte: " + (descarte[-1] if len(descarte) > 0 else "(vacío)"))

    if tiene_trio(mano_pc):
        print("(PC) ¡Ya tenía TRÍO!")
        return True

    # (1) Robo: si la carta del descarte ayuda a par/trío, tomarla; si no, mazo
    tomo = False
    if len(descarte) > 0:
        vtop = obtener_valor(descarte[-1])
        cnt = 0
        for c in mano_pc:
            if obtener_valor(c) == vtop:
                cnt += 1
        if cnt >= 1:
            robar_de_descarte(mano_pc, descarte)
            print("(PC) Toma del descarte: " + mano_pc[-1])
            tomo = True

    if not tomo:
        if robar_de_mazo(mano_pc, mazo):
            print("(PC) Roba del mazo: " + mano_pc[-1])
        else:
            print("(PC) El mazo está vacío.")

    # (2) ¿Ganó al robar?
    if tiene_trio(mano_pc):
        print("(PC) ¡Formó un TRÍO!")
        return True

    # (3) Descartar el valor menos frecuente (conserva pares)
    valores, conteos = contar_valores(mano_pc)
    if len(valores) > 0:
        minc = conteos[0]
        vmin = valores[0]
        for k in range(1, len(valores)):
            if conteos[k] < minc:
                minc = conteos[k]
                vmin = valores[k]
        # descartar la primera carta cuyo valor sea vmin
        for i in range(len(mano_pc)):
            if obtener_valor(mano_pc[i]) == vmin:
                carta = mano_pc[i]
                descartar(mano_pc, descarte, i)
                print("(PC) Descarta: " + carta)
                break

    return False

# -----------------
# Orquestación
# -----------------

def jugar_rummy():
    print("=== RUMMY (TRÍO) — Versión simplificada ===")
    mazo = crear_mazo()

    # Reparto inicial: 3 y 3, 1 al descarte
    jugador = []
    computadora = []
    for _ in range(3):
        jugador.append(mazo.pop())
        computadora.append(mazo.pop())
    descarte = [ mazo.pop() ]

    print("Tu mano inicial: " + mano_a_str(jugador))
    print("Descarte inicial: " + descarte[-1])

    # Bucle de turnos
    while True:
        if turno_jugador(jugador, mazo, descarte):
            print("\n>>> ¡GANASTE! Formaste un TRÍO antes que la PC.")
            break
        if turno_computadora(computadora, mazo, descarte):
            print("\n>>> La PC gana. Formó un TRÍO.")
            break

def main():
    jugar_rummy()

if __name__ == "__main__":
    main()


=== RUMMY (TRÍO) — Versión simplificada ===
Tu mano inicial: [0] 5♦, [1] Q♥, [2] 2♣
Descarte inicial: Q♦

--- TU TURNO ---
Tu mano: [0] 5♦, [1] Q♥, [2] 2♣
Tope descarte: Q♦
